In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/ratings_small.csv


In [42]:
df=pd.read_csv('../input/the-movies-dataset/movies_metadata.csv',low_memory=False)
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Assigning score to each movie based on ratings and number of reviews

In [43]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Checking for NULL values

In [44]:
df.isna().any()

adult                    False
belongs_to_collection     True
budget                   False
genres                   False
homepage                  True
id                       False
imdb_id                   True
original_language         True
original_title           False
overview                  True
popularity                True
poster_path               True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
status                    True
tagline                   True
title                     True
video                     True
vote_average              True
vote_count                True
dtype: bool

 we will use two parameters number of votes and rating of a movie to assign score

In [45]:
df[['vote_count','vote_average']].head()

,vote_count,vote_average
0,5415.0,7.7
1,2413.0,6.9
2,92.0,6.5
3,34.0,6.1
4,173.0,5.7


weighted average rating = ( vote_average * vote_count+mean(vote_average) * threshhold)/(vote_count+threshhold))

threshold= minimum number of vote_count for a movie to assign a score

vote_average= rating


In [46]:
threshold=df['vote_count'].quantile(0.9)
print(threshold)

160.0


In [47]:
meanvote=df['vote_average'].mean()
print(meanvote)

5.618207215133889


In [48]:
#calculating score for all movies
moviescore=[]
for voteval in range(len(df['vote_count'])):
    if(voteval>threshold):
        moviescore.append((df['vote_average'][voteval]*df['vote_count'][voteval]+meanvote*threshold)/(df['vote_count'][voteval]+threshold))
    else:
        moviescore.append(0)

In [49]:
df['moviescore']=moviescore

The movie score tells us how good a movie is based on user rating and number of ratings

Now we have a recommneder system which giver us top 25 movies among the given movies. These movies will be liked by most of the users so for a new user we can suggest these movies to start

In [50]:
df[['title','vote_average','vote_count','moviescore']].sort_values('moviescore',ascending=False).head(25)

,title,vote_average,vote_count,moviescore
314,The Shawshank Redemption,8.5,8358.0,8.445869
834,The Godfather,8.5,6024.0,8.425439
10309,Dilwale Dulhania Le Jayenge,9.1,661.0,8.421453
12481,The Dark Knight,8.3,12269.0,8.265477
2843,Fight Club,8.3,9678.0,8.256385
292,Pulp Fiction,8.3,8670.0,8.251406
522,Schindler's List,8.3,4436.0,8.206639
23673,Whiplash,8.3,4376.0,8.205404
5481,Spirited Away,8.3,3968.0,8.196055
2211,Life Is Beautiful,8.3,3643.0,8.187171


Finding similarity between movies

In [51]:
df[['title','overview','genres']].head()

,title,overview,genres
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]"


# **content based filtering **

We will use cosine similarity to compare two movies

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

we will use movie overview to calculate similarities between two movies

In [53]:
df['overview'].isna().any()

True

Replace null values in the overview column with empty strings

In [54]:
df['overview']=df['overview'].fillna('')

In [55]:
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape

(45466, 75827)

In [56]:
df.shape

(45466, 25)

In [57]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix[:10000], tfidf_matrix[:10000])
cosine_sim.shape

(10000, 10000)

Suggesting movies based on cosine similarirty

In [58]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices[:5]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [59]:
def suggest_movies(movie_title):
    movieindex=list(enumerate(cosine_sim[indices[movie_title]]))
    sim_scores = sorted(movieindex, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    print(df['title'].iloc[movie_indices])
    
    

In [60]:
suggest_movies('Toy Story')

2997              Toy Story 2
8327                The Champ
1071    Rebel Without a Cause
1932                Condorman
3057          Man on the Moon
485                    Malice
1032            The Sunchaser
3252          Bound for Glory
7254           Africa Screams
2157        Indecent Proposal
Name: title, dtype: object


#   Collabrative based filtering 

In [61]:
rdf=pd.read_csv('../input/the-movies-dataset/ratings_small.csv')
rdf.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [62]:
rdf=rdf[['userId','movieId','rating']]
rdf.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


collabrative user- user based interaction

Building a user item matrix

In [63]:
useritem = np.zeros([10000, 10000])
for userid in range(len(rdf['userId'])):
    try:
        useritem[rdf['userId'][userid]-1][rdf['movieId'][userid]-1]=rdf['rating'][userid]
    except:
        pass
        

In [64]:
moviecolumns=[]
for movieval in range(len(useritem)):
    moviecolumns.append("movie "+str(movieval))

In [65]:
mdf=pd.DataFrame(useritem,columns=moviecolumns)

ValueError: Shape of passed values is (1000, 100000), indices imply (1000, 1000)

In [ ]:
mdf.head()

In [ ]:
mdf['movie 30'][0]

In [ ]:
def standerdize(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row

In [ ]:
smdf=mdf.apply(standerdize)
smdf=smdf.fillna(0)
smdf.head()


item-item based collabrative filtering

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
item_similarity=cosine_similarity(smdf.T)

In [ ]:
its=pd.DataFrame(item_similarity,columns=moviecolumns)

In [ ]:
its.head()

In [ ]:
def suggest_movies(movie_name,user_rating):
    similar_score=its[movie_name]*(user_rating-rdf['rating'].mean())
    similar_score=similar_score.sort_values(ascending=False)
    return similar_score[:5]

Suggesting movies based on user past movie reviews

In [ ]:
suggest_movies('movie 0',5)

# Using SVD to predict rating a user would assign to a movie

In [ ]:
import surprise
from surprise import Dataset, SVD 
from surprise.model_selection import cross_validate
reader = surprise.reader.Reader()
data = Dataset.load_from_df(rdf[['userId', 'movieId', 'rating']], reader)
svd=SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Now we can predict a rating which a user will give for a movie

In [ ]:
svd.predict(2,531)

User 2 will assign a rating of 3.584 for a movie with id 531